In [ ]:
import socket

class spark_sql_proxy_client:
    def __init__(self, sock=None):
        if sock is None:
            self.sock = socket.socket(
                socket.AF_INET, socket.SOCK_STREAM)
        else:
            self.sock = sock

    def connect(self, host, port):
        self.sock.connect((host, port))

    def send(self, msg):
        totalsent = 0
        while totalsent < len(msg):
            sent = self.sock.send(msg[totalsent:])
            if sent == 0:
                raise RuntimeError("socket connection broken")
            totalsent = totalsent + sent

    def readlines(self, recv_buffer=4096, delim='\n'):
      buffer = ''
      data = True
      while data:
        data = self.sock.recv(recv_buffer)
        buffer += data

        while buffer.find(delim) != -1:
            line, buffer = buffer.split('\n', 1)
            yield line
        return


client = spark_sql_proxy_client()
client.connect('104.197.137.27', 12345)
client.send("create temporary table people_json3 using org.apache.spark.sql.json options (path 'gs://alekseyv-test/people.json')\n")
client.readlines()
for line in client.readlines():
  print(line)
client.send("select * from people_json3\n")
client.readlines()
for line in client.readlines():
  print(line)

  